In [7]:
from uesgraphs.uesgraph import UESGraph


In [8]:


json = r"D:\rka-lko\git\uesgraphs\workspace\e11\inputs\test_modelgen\Ibpsa\nodes.json"
json = r"D:\rka-lko\git\uesgraphs\workspace\e11\inputs\test_modelgen\Pinola\nodes.json"
json = "/home/leon/git/uesgraphs_github/uesgraphs/workspace/e10/tests/test_modelgen/Ibpsa/"
graph = UESGraph()

In [9]:
graph.from_json(path=json,network_type="heating")

read nodes...
******
 input_ids were {'buildings': None, 'nodes': 'f8a0e23d-465f-41f2-918e-4afbd5994b9e', 'pipes': None, 'supplies': None}
...finished


In [10]:
#import uesgraphs.systemmodels.utilities as ut

In [11]:
#graph2 = ut.estimate_m_flow_nominal(graph,30,"heating")

In [12]:
import tempfile
import os
from datetime import datetime
import logging

import pandas as pd

import warnings
from typing import Any, Dict, List, Optional, Set, Tuple

import networkx as nx



In [25]:
graph.nodes[1003]

{'name': 'point_3',
 'node_type': 'building',
 'position': <POINT (0 0)>,
 'is_supply_heating': False,
 'is_supply_cooling': False,
 'is_supply_electricity': False,
 'is_supply_gas': False,
 'is_supply_other': False,
 'input_heat': [3640.07987448,
  3514.56012552,
  3891.11987448,
  3640.07987448,
  4393.2,
  4393.2,
  3640.07987448,
  4142.16,
  3891.11987448,
  4142.16,
  3891.11987448,
  3891.11987448,
  4142.16,
  4016.6402510400003,
  3891.11987448,
  3891.11987448,
  3640.07987448,
  4267.68025104,
  4267.68025104,
  3891.11987448,
  4142.16,
  10418.159623440002,
  8284.32,
  8033.28037656,
  8033.28037656,
  13305.119999999999,
  1882.7999999999997,
  7782.23987448,
  7405.68,
  7405.68,
  7405.68,
  7029.12025104,
  7405.68,
  6150.479874480001,
  7405.68,
  5773.919999999999,
  5773.919999999999,
  6527.0401255199995,
  5522.879999999999,
  5397.36012552,
  5271.839874480001,
  4518.71974896,
  4518.71974896,
  4393.2,
  4769.759874480001,
  5146.32012552,
  4644.24,
  4518.7

In [27]:
size_hydronic_network(graph,dT_attribute="dT_design",catalog="isoplus")

Logfile findable here: /tmp/hydronic_network_sizing_20250523_195016.log


NameError: name '__file__' is not defined

In [22]:

def size_hydronic_network(
    graph: Any,
    m_flow_key = None,
    catalog = None,
    dT_attribute: str = "dT_Network",
    network_type: str = "heating",
    demand_attribute: str = "input_heat",
    load_scenario: str = "peak_load",
    cp: float = 4000,
    logger: Optional[logging.Logger] = None
) -> Any:
    
    # Set up logger
    if logger is None:
        logger = set_up_logger(
            name="hydronic_network_sizing",
            #log_dir="./logs",  # Optional: specify custom directory
            level=logging.DEBUG  # INFO level captures all important steps
        )

    if m_flow_key == None:
        # Estimate mass flows
        logger.info(f"Sizing hydronic network: estimating mass flows for {network_type} network with {load_scenario} scenario")
        graph = estimate_m_flow_demand_based(
            graph=graph,
            network_type=network_type,
            demand_attribute=demand_attribute,
            load_scenario=load_scenario,
            cp=cp,
            dT_attribute=dT_attribute,
            logger=logger  
        )
        m_flow_key = f"m_flow_{load_scenario}"

    logger.info("Mass flow estimation completed successfully")
    
    # Estimate pipe diameters
    logger.info(f"Sizing hydronic network: estimating pipe diameters for {network_type} network with {load_scenario} scenario")
    if catalog:
        df = load_pipe_catalog(catalog)
        get_pipe_catalog_DN_m_flow(graph,pipe_catalog=df,logger=logger, mass_flow_key=m_flow_key, dn_key="DN", diameter_key="diameter", robust=True)
    else:
        raise NotImplementedError()
    
    return graph



In [14]:

def get_pipe_catalog_DN_m_flow(
    graph,
    pipe_catalog: pd.DataFrame,
    logger: logging.Logger,
    mass_flow_key: str,
    dn_key: str,
    diameter_key: str,
    robust: bool = True
) -> None:
    # Collect all edges missing the required mass flow attribute
    missing_edges = []
    for edge in graph.edges:
        if mass_flow_key not in graph.edges[edge]:
            missing_edges.append(edge)

    if missing_edges:
        logger.error("The following edges are missing the '%s' attribute:", mass_flow_key)
        for edge in missing_edges:
            logger.error("  - %s", edge)
        raise ValueError(
            f"Aborting because edges are missing the '{mass_flow_key}' attribute."
        )

    # Update DN and the diameter information for each edge
    for edge in graph.edges:
        m_flow = graph.edges[edge][mass_flow_key]
        matching_rows = pipe_catalog[
            (pipe_catalog['mass_flow_min'] <= m_flow) &
            (pipe_catalog['mass_flow_max'] >= m_flow)
        ]
        num_matches = len(matching_rows)

        # Scenario 1: No matching row
        if num_matches == 0:
            if robust:
                # Attempt to find the next bigger pipe
                bigger_rows = pipe_catalog[pipe_catalog['mass_flow_min'] > m_flow]
                if not bigger_rows.empty:
                    logger.warning(
                        "Edge %s: No row directly matches mass_flow=%.2f. "
                        "Using the next bigger pipe.",
                        edge, m_flow
                    )
                    next_bigger_idx = bigger_rows['mass_flow_min'].idxmin()
                    chosen_row = bigger_rows.loc[next_bigger_idx]
                else:
                    # Fallback: there is no bigger pipe either
                    raise ValueError(
                        f"Edge {edge}: No direct match for flow {m_flow}, "
                        "and no bigger pipe available. Consider extending your pipe catalog."
                    )
            else:
                raise ValueError(
                    f"Edge {edge}: No matching row for flow {m_flow}, and robust=False. "
                    "Consider adding more pipe entries or adjusting your data."
                )

        # Scenario 2: Exactly one match
        elif num_matches == 1:
            chosen_row = matching_rows.iloc[0]

        # Scenario 3: Multiple matching rows
        else:
            if robust:
                logger.warning(
                    "Edge %s: Multiple matching rows for mass_flow=%.2f. "
                    "Selecting the row with the largest DN among them.",
                    edge, m_flow
                )
                max_dn_idx = matching_rows['DN'].idxmax()
                chosen_row = matching_rows.loc[max_dn_idx]
            else:
                raise ValueError(
                    f"Edge {edge}: Multiple rows match flow {m_flow}, robust=False. "
                    "Validate pipe catalog ranges for overlaps or remove duplicates."
                )

        dn_value = chosen_row['DN']
        diameter_value = float(chosen_row['inner_diameter'])

        if dn_key in graph.edges[edge]:
            logger.warning(
                "Edge %s already has '%s' set to %s and will be overwritten.",
                edge, dn_key, graph.edges[edge][dn_key]
            )
        if diameter_key in graph.edges[edge]:
            logger.warning(
                "Edge %s already has '%s' set to %s and will be overwritten.",
                edge, diameter_key, graph.edges[edge][diameter_key]
            )

        graph.edges[edge][dn_key] = dn_value
        graph.edges[edge][diameter_key] = diameter_value

        logger.info(
            "Edge %s updated: %s=%s, %s=%.2f (mass_flow=%.2f)",
            edge, dn_key, dn_value, diameter_key, diameter_value, m_flow
        )

In [15]:

def load_pipe_catalog(catalog_name: str = "isoplus") -> pd.DataFrame:
    """
    Load pipe catalog data from CSV file in the data/pipe_catalogs directory.
    
    This function loads manufacturer pipe catalog data containing pipe dimensions
    and flow capacities for different nominal diameters (DN). The catalog files
    are expected to be located in the data/pipe_catalogs subdirectory relative
    to the systemmodels module.
    
    Parameters
    ---------- 
    catalog_name : str, optional
        Name of the pipe catalog to load (default: "isoplus")
        The function will look for a file named "{catalog_name}.csv"
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing pipe catalog data with columns:
        - DN: Nominal diameter [mm]
        - wall_thickness: Pipe wall thickness [mm] 
        - inner_diameter: Inner pipe diameter [mm]
        - mass_flow_min: Minimum mass flow capacity [t/h]
        - mass_flow_max: Maximum mass flow capacity [t/h]
        
    Raises
    ------
    FileNotFoundError
        If the specified catalog file does not exist
    ValueError
        If the catalog file exists but contains invalid data structure
        
    Examples
    --------
    >>> catalog = load_pipe_catalog("isoplus")
    >>> print(catalog.columns.tolist())
    ['DN', 'wall_thickness', 'inner_diameter', 'mass_flow_min', 'mass_flow_max']
    
    >>> # Load different catalog (if available)
    >>> rehau_catalog = load_pipe_catalog("rehau")
    
    Notes
    -----
    The CSV files can contain comment lines starting with '#' which will be
    automatically ignored during loading. This allows for metadata and source
    information to be stored directly in the catalog files.
    
    The function expects the catalog files to be located at:
    {module_directory}/uesgraphs/data/pipe_catalogs/{catalog_name}.csv
    """
    # Construct path to catalog file relative to this module
    #current_dir = os.path.dirname(__file__)
    current_dir = os.getcwd()  # Get the current working directory
    catalog_path = os.path.join(
        current_dir, 
        "..", 
        "data", 
        "pipe_catalogs", 
        f"{catalog_name}.csv"
    )
    
    # Convert to absolute path for better error reporting
    catalog_path = os.path.abspath(catalog_path)
    
    # Check if catalog file exists
    if not os.path.isfile(catalog_path):
        available_catalogs = _get_available_catalogs()
        raise FileNotFoundError(
            f"Pipe catalog '{catalog_name}' not found at: {catalog_path}\n"
            f"Available catalogs: {available_catalogs}"
        )
    
    try:
        # Load CSV data, ignoring comment lines starting with '#'
        catalog_df = pd.read_csv(catalog_path, comment='#')
        
        # Validate required columns exist
        required_columns = ['DN', 'inner_diameter', 'mass_flow_min', 'mass_flow_max']
        missing_columns = [col for col in required_columns if col not in catalog_df.columns]
        
        if missing_columns:
            raise ValueError(
                f"Catalog '{catalog_name}' is missing required columns: {missing_columns}\n"
                f"Available columns: {catalog_df.columns.tolist()}"
            )
        
        # Sort by DN for consistent ordering
        catalog_df = catalog_df.sort_values('inner_diameter').reset_index(drop=True)
        
        return catalog_df
        
    except pd.errors.EmptyDataError:
        raise ValueError(f"Catalog file '{catalog_name}' is empty or contains no valid data")
    except pd.errors.ParserError as e:
        raise ValueError(f"Error parsing catalog file '{catalog_name}': {str(e)}")


def _get_available_catalogs() -> list:
    """
    Get list of available pipe catalog files.
    
    Returns
    -------
    list
        List of available catalog names (without .csv extension)
    """
    try:
        current_dir = os.path.dirname(__file__)
        catalog_dir = os.path.join(current_dir, "..", "data", "pipe_catalogs")
        catalog_dir = os.path.abspath(catalog_dir)
        
        if not os.path.isdir(catalog_dir):
            return []
            
        # Get all CSV files in catalog directory
        csv_files = [f for f in os.listdir(catalog_dir) if f.endswith('.csv')]
        # Remove .csv extension to get catalog names
        catalog_names = [os.path.splitext(f)[0] for f in csv_files]
        
        return sorted(catalog_names)
        
    except (OSError, IOError):
        return []


def get_catalog_info(catalog_name: str = "isoplus") -> dict:
    """
    Get basic information about a pipe catalog.
    
    Parameters
    ----------
    catalog_name : str, optional
        Name of the pipe catalog (default: "isoplus")
        
    Returns
    -------
    dict
        Dictionary containing catalog information:
        - name: Catalog name
        - num_sizes: Number of available pipe sizes
        - dn_range: Tuple of (min_DN, max_DN)
        - flow_range: Tuple of (min_flow, max_flow) in t/h
        
    Examples
    --------
    >>> info = get_catalog_info("isoplus")
    >>> print(f"Catalog has {info['num_sizes']} pipe sizes")
    >>> print(f"DN range: {info['dn_range'][0]} - {info['dn_range'][1]} mm")
    """
    catalog_df = load_pipe_catalog(catalog_name)
    
    return {
        'name': catalog_name,
        'num_sizes': len(catalog_df),
        'dn_range': (catalog_df['DN'].min(), catalog_df['DN'].max()),
        'flow_range': (catalog_df['mass_flow_min'].min(), catalog_df['mass_flow_max'].max()),
        'diameter_range': (catalog_df['inner_diameter'].min(), catalog_df['inner_diameter'].max())
    }

In [16]:
edges = list(updated_graph.edges)
for edge in edges:
    print(f"{edge[0]} {edge[1]} with {updated_graph.edges[edge]["m_flow_peak_load"]} m3/h")


NameError: name 'updated_graph' is not defined

In [17]:

def set_up_logger(name, log_dir=None, level=int(logging.INFO)):  # Changed to INFO for more details
    """
    Set up a file-based logger with timestamp and detailed formatting.
    
    Parameters
    ----------
    name : str
        Logger name, used for log file naming
    log_dir : str, optional
        Directory for log files. If None, uses system temp directory
    level : int, optional
        Logging level (default: INFO for detailed mass flow logging)
        
    Returns
    -------
    logging.Logger
        Configured logger instance writing to timestamped file
    """
    logger = logging.getLogger(name)
    logger.setLevel(level)
    
    # Determine log directory
    if log_dir is None:
        log_dir = tempfile.gettempdir()
    
    # Create timestamped log file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = os.path.join(log_dir, f"{name}_{timestamp}.log")
    print(f"Logfile findable here: {log_file}")
    
    # Configure file handler with detailed formatting
    handler = logging.FileHandler(log_file)
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - [%(filename)s:%(lineno)d] - %(levelname)s - %(message)s'
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger



In [19]:

def estimate_m_flow_demand_based(
    graph: Any,
    network_type: str = "heating",
    demand_attribute: str = "input_heat",
    load_scenario: str = "peak_load",
    cp: float = 4000,
    dT_attribute: str = "dT_Network",
    logger: Optional[logging.Logger] = None
) -> Any:
    """
    Estimates mass flow for each edge by calculating flows at demand nodes and propagating backwards.
    
    This function implements a physically correct approach where mass flows are calculated
    at each demand node based on their specific load and temperature difference, then
    aggregated backwards through the network following mass conservation principles.
    
    Parameters
    ----------
    graph : UESGraph or nx.Graph
        The graph representing the network with nodelist_building attribute.
    network_type : str, optional
        Type of network, default is "heating". Must be "heating" or "cooling".
    demand_attribute : str, optional
        Attribute name containing load values in demand nodes, default is "input_heat".
    load_scenario : str, optional
        Load scenario for calculation, default is "peak_load".
        Options: "peak_load" (maximum value) or "average_load" (mean value).
    cp : float, optional
        Specific heat capacity of the fluid in J/(kg*K), default is 4000.
    dT_attribute : str
        Node attribute name for temperature difference values in Kelvin.
        Must be present in all demand nodes with positive numeric values.
        Used for individual mass flow calculations at each demand node.
    logger : logging.Logger, optional
        Logger instance for logging messages. If None, creates a default logger.
        
    Returns
    -------
    graph : UESGraph or nx.Graph
        Input graph with additional edge attributes:
        - m_flow_{load_scenario}: Mass flow rate in kg/s for each edge
        - contributing_demands_{load_scenario}: List of demand nodes contributing to edge flow
        - supply_attribution_{load_scenario}: Dictionary showing which supply serves which demands
        
    Raises
    ------
    TypeError
        If graph does not have required nodelist_building attribute.
    ValueError
        If network_type or load_scenario parameters are invalid.
        If no supply or demand nodes are found.
        If required node attributes are missing.
    """
    # Initialize logger
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Validate input parameters
    _validate_parameters(graph, network_type, load_scenario, demand_attribute)
    
    # Step 1: Identify and categorize nodes
    supply_nodes, demand_nodes = _identify_network_nodes(
        graph, network_type, demand_attribute, logger
    )
    
    # Step 2: Calculate mass flows at demand nodes
    demand_mass_flows = _calculate_demand_mass_flows(
        graph, demand_nodes, demand_attribute, load_scenario, cp, dT_attribute, logger
    )
    
    # Step 3: Build supply-to-demand flow paths
    supply_demand_paths = _build_flow_paths(
        graph, supply_nodes, demand_nodes, logger
    )
    
   # Step 4: Aggregate mass flows on edges using maximum principle
    _aggregate_edge_flows_robust(
        graph, supply_demand_paths, demand_mass_flows, load_scenario, logger
    )
    
    logger.info(f"Successfully calculated mass flows for {len(graph.edges)} edges "
               f"using demand-based approach with {load_scenario} scenario")
    
    return graph


def _validate_parameters(
    graph: Any, 
    network_type: str, 
    load_scenario: str, 
    demand_attribute: str
) -> None:
    """Validate input parameters for the mass flow estimation function."""
    if not hasattr(graph, "nodelist_building"):
        raise TypeError("Graph must be a UESGraph object with nodelist_building attribute")
    
    if network_type not in ["heating", "cooling"]:
        raise ValueError("network_type must be 'heating' or 'cooling'")
    
    if load_scenario not in ["peak_load", "average_load"]:
        raise ValueError("load_scenario must be 'peak_load' or 'average_load'")


def _identify_network_nodes(
    graph: Any, 
    network_type: str, 
    demand_attribute: str, 
    logger: logging.Logger
) -> Tuple[List[Any], List[Any]]:
    """
    Identify and categorize supply and demand nodes in the network.
    
    Returns
    -------
    tuple
        (supply_nodes, demand_nodes) - Lists of supply and demand node identifiers
    """
    supply_nodes = []
    demand_nodes = []
    supply_attr = f"is_supply_{network_type}"
    
    for node in graph.nodelist_building:
        # Check for required supply attribute
        if supply_attr not in graph.nodes[node]:
            raise ValueError(f"Node {node} missing required attribute '{supply_attr}'")
        
        if graph.nodes[node][supply_attr]:
            supply_nodes.append(node)
        else:
            # Validate demand node has required load attribute
            if demand_attribute not in graph.nodes[node]:
                raise ValueError(f"Demand node {node} missing required attribute '{demand_attribute}'")
            demand_nodes.append(node)
    
    # Ensure we have both supply and demand nodes
    if not supply_nodes:
        raise ValueError(f"No supply nodes found for network type '{network_type}'")
    if not demand_nodes:
        raise ValueError(f"No demand nodes found for network type '{network_type}'")
    
    logger.info(f"Identified {len(supply_nodes)} supply nodes and {len(demand_nodes)} demand nodes")
    return supply_nodes, demand_nodes


def _calculate_demand_mass_flows(
    graph: Any,
    demand_nodes: List[Any],
    demand_attribute: str,
    load_scenario: str,
    cp: float,
    dT_attribute: str,
    logger: logging.Logger
) -> Dict[Any, float]:
    """
    Calculate mass flow requirements at each demand node.
    
    This function processes each demand node individually, calculating the required
    mass flow based on the node's load and temperature difference (dT).
    
    Parameters
    ----------
    graph : UESGraph or nx.Graph
        Network graph containing node data
    demand_nodes : List[Any]
        List of demand node identifiers
    demand_attribute : str
        Attribute name containing load values
    load_scenario : str
        Either "peak_load" or "average_load"
    cp : float
        Specific heat capacity in J/(kg*K)
    dT_attribute : str
        Node attribute name for temperature difference values in Kelvin.
        Must be present in all demand nodes with positive numeric values.
        Used for individual mass flow calculations at each demand node.
    logger : logging.Logger
        Logger for status messages
        
    Returns
    -------
    Dict[Any, float]
        Dictionary mapping demand node identifiers to their mass flow requirements in kg/s
    """
    
    for demand_node in demand_nodes:
        if dT_attribute not in graph.nodes[demand_node]:
            # Get available attributes for better error message
            available_attrs = list(graph.nodes[demand_node].keys())
            dT_related_attrs = [attr for attr in available_attrs if 'dT' in attr or 'delta' in attr.lower() or 'temp' in attr.lower()]

            error_msg = (
                f"Demand node '{demand_node}' is missing the required temperature difference attribute '{dT_attribute}'. "
                f"This attribute must contain the temperature difference (dT) in Kelvin between supply and return flow "
                f"for mass flow calculation (formula: m_flow = thermal_load / (cp * dT)).\n\n"
                f"To fix this issue:\n"
                f"1. Add '{dT_attribute}' attribute to node '{demand_node}' with a numeric value in Kelvin\n, like with graph.nodes[{demand_node}]['{dT_attribute}'] = 30\n"
                f"2. Or specify a different attribute name using the 'dT_attribute' parameter\n"
            )

            if dT_related_attrs:
                error_msg += f"\nConsider using: dT_attribute='{dT_related_attrs[0]}' if appropriate, when calling method"

            raise ValueError(error_msg)
        
    demand_mass_flows = {}
    
    for demand_node in demand_nodes:
        # Extract load values from node attribute
        load_values = graph.nodes[demand_node][demand_attribute]
        
        # Handle both single values and lists
        if isinstance(load_values, (int, float)):
            load_values = [load_values]
        
        # Convert to absolute values for calculation
        abs_load_values = [abs(x) for x in load_values]
        
        # Calculate load based on scenario
        if load_scenario == "peak_load":
            load = max(abs_load_values)
        elif load_scenario == "average_load":
            load = sum(abs_load_values) / len(abs_load_values)
        
        # Get node-specific temperature difference
        node_dT = graph.nodes[demand_node][dT_attribute]
        # Validate dT value
        if not isinstance(node_dT, (int, float)) or node_dT <= 0:
            raise ValueError(
                f"Temperature difference (dT) for demand node '{demand_node}' must be a positive number, "
                f"got {node_dT} (type: {type(node_dT).__name__})"
            )

        # Calculate mass flow: m_flow = Q / (cp * dT)
        mass_flow = load / (cp * node_dT)
        demand_mass_flows[demand_node] = mass_flow
        
        logger.debug(f"Demand node {demand_node}: load={load:.2f}W, dT={node_dT}K, "
                    f"m_flow={mass_flow:.6f}kg/s")
    
    
    total_demand_flow = sum(demand_mass_flows.values())
    logger.info(f"Calculated mass flows for {len(demand_nodes)} demand nodes, "
               f"total demand: {total_demand_flow:.6f} kg/s")
    
    return demand_mass_flows

def _build_flow_paths(
    graph: Any,
    supply_nodes: List[Any],
    demand_nodes: List[Any],
    logger: logging.Logger
) -> Dict[Tuple[Any, Any], List[Tuple[Any, Any]]]:
    """
    Build flow paths from each supply node to reachable demand nodes.
    
    This function identifies all supply-demand pairs that are connected and
    determines the shortest path between them, converting paths to edge lists.
    
    Parameters
    ----------
    graph : UESGraph or nx.Graph
        Network graph
    supply_nodes : List[Any]
        List of supply node identifiers
    demand_nodes : List[Any]
        List of demand node identifiers
    logger : logging.Logger
        Logger for status messages
        
    Returns
    -------
    Dict[Tuple[Any, Any], List[Tuple[Any, Any]]]
        Dictionary mapping (supply, demand) tuples to lists of edges in the flow path
    """
    supply_demand_paths = {}
    unreachable_demands = set(demand_nodes)  # Track which demands are reachable
    
    for supply in supply_nodes:
        reachable_from_this_supply = []
        
        for demand in demand_nodes:
            try:
                if nx.has_path(graph, supply, demand):
                    # Calculate shortest path and convert to edge list
                    node_path = nx.shortest_path(graph, supply, demand)
                    edge_path = [(node_path[i], node_path[i + 1]) 
                                for i in range(len(node_path) - 1)]
                    
                    supply_demand_paths[(supply, demand)] = edge_path
                    reachable_from_this_supply.append(demand)
                    
                    # Remove from unreachable set
                    unreachable_demands.discard(demand)
                    
            except nx.NetworkXNoPath:
                # Explicitly handle case where no path exists
                continue
        
        logger.debug(f"Supply {supply} can reach {len(reachable_from_this_supply)} demands: "
                    f"{reachable_from_this_supply}")
    
    # Log summary information
    total_paths = len(supply_demand_paths)
    logger.info(f"Built {total_paths} supply-to-demand flow paths")
    
    if unreachable_demands:
        logger.warning(f"Unreachable demand nodes found: {list(unreachable_demands)}")
    
    return supply_demand_paths

def _aggregate_edge_flows_robust(
    graph: Any,
    supply_demand_paths: Dict[Tuple[Any, Any], List[Tuple[Any, Any]]],
    demand_mass_flows: Dict[Any, float],
    load_scenario: str,
    logger: logging.Logger
) -> None:
    """
    Aggregate mass flows on edges using a robust supply-based approach with maximum flow principle.
    
    This function implements a two-stage aggregation process:
    1. For each supply node, calculate cumulative flows on all edges serving its connected demands
    2. Apply maximum flow principle when multiple supplies can serve the same edge
    
    This approach ensures robust network sizing where each edge is dimensioned for the worst-case
    scenario among all possible supply configurations.
    
    Parameters
    ----------
    graph : UESGraph or nx.Graph
        Network graph to be modified with calculated flow data
    supply_demand_paths : Dict[Tuple[Any, Any], List[Tuple[Any, Any]]]
        Mapping of (supply, demand) pairs to their corresponding edge paths
    demand_mass_flows : Dict[Any, float]
        Mass flow requirements for each demand node in kg/s
    load_scenario : str
        Load scenario identifier used for attribute naming in the graph
    logger : logging.Logger
        Logger instance for progress and summary information
    """
    logger.info("Starting robust edge flow aggregation using supply-based approach")
    
    # Initialize data structures for supply-based flow tracking
    supply_edge_flows = {}  # {supply_node: {edge: accumulated_flow}}
    edge_contributing_demands = {}  # {edge: set_of_demand_nodes}
    supply_attribution = {}  # {demand_node: supply_node}
    
    # Step 1: Group supply-demand paths by supply node for separate processing
    supplies = set(supply for supply, demand in supply_demand_paths.keys())
    logger.info(f"Processing flows for {len(supplies)} supply nodes")
    
    # Step 2: Calculate aggregated flows for each supply node independently
    for supply in supplies:
        supply_edge_flows[supply] = {}
        demands_served = []
        
        logger.debug(f"Processing supply node: {supply}")
        
        # Process all demand nodes served by this supply
        for (sup, demand), edge_path in supply_demand_paths.items():
            if sup == supply:
                demand_flow = demand_mass_flows[demand]
                demands_served.append(demand)
                
                # Record supply attribution for this demand
                supply_attribution[demand] = supply
                
                # Accumulate demand flow along the entire path to this demand
                for edge in edge_path:
                    # Add flow to supply-specific edge flow tracking
                    if edge in supply_edge_flows[supply]:
                        supply_edge_flows[supply][edge] += demand_flow
                    else:
                        supply_edge_flows[supply][edge] = demand_flow
                    
                    # Track which demands contribute to each edge for documentation
                    if edge not in edge_contributing_demands:
                        edge_contributing_demands[edge] = set()
                    edge_contributing_demands[edge].add(demand)
        
        total_supply_flow = sum(demand_mass_flows[d] for d in demands_served)
        logger.debug(f"Supply {supply}: serves {len(demands_served)} demands, "
                    f"total flow = {total_supply_flow:.6f} kg/s")
    
    logger.debug(f"Identified flows: {supply_edge_flows}")

    # Step 3: Apply maximum flow principle across all supplies for robust sizing
    logger.info("Applying maximum flow principle across supplies for robust edge sizing")
    final_edge_flows = {}
    supply_conflicts = {}  # Track edges with multiple supply options
    
    for supply, edge_flows in supply_edge_flows.items():
        for edge, flow in edge_flows.items():
            # Normalize edge representation for consistent dictionary access
            normalized_edge = __normalize_edge(edge)
            logger.debug(f"Processing edge {edge} with flow {flow:.6f} kg/s from supply {supply}")
            if normalized_edge  in final_edge_flows:
                # Multiple supplies can serve this edge - apply maximum principle
                if normalized_edge  not in supply_conflicts:
                    supply_conflicts[normalized_edge ] = []
                supply_conflicts[normalized_edge ].append((supply, flow))
                
                # Update to maximum flow value for robust design
                final_edge_flows[normalized_edge ] = max(final_edge_flows[normalized_edge ], flow)
            else:
                # First supply to use this edge
                final_edge_flows[normalized_edge ] = flow
    
    # Log information about supply conflicts and robust sizing decisions
    if supply_conflicts:
        logger.info(f"Applied maximum flow principle to {len(supply_conflicts)} edges "
                   f"with multiple supply options")
        for edge, conflicts in supply_conflicts.items():
            flows = [f"{supply}: {flow:.6f}" for supply, flow in conflicts]
            max_flow = final_edge_flows[edge]
            logger.debug(f"Edge {edge} - Supplies: [{', '.join(flows)}] → "
                        f"Selected: {max_flow:.6f} kg/s")
    
    # Step 4: Apply calculated flows to all graph edges
    edges_with_flow = 0
    edges_without_flow = 0
    
    for edge in graph.edges:
        # Normalize the current graph edge for dictionary lookup
        normalized_edge = __normalize_edge(edge)
        if normalized_edge in final_edge_flows:
            # Set mass flow attribute for edges with calculated flows
            graph.edges[normalized_edge][f"m_flow_{load_scenario}"] = final_edge_flows[normalized_edge]
            
            
            edges_with_flow += 1
        else:
            logger.warning(f"Edge {edge} has no flow assigned, setting to 0.0 kg/s")
            # Initialize edges without flow (not part of any supply-demand path)
            graph.edges[normalized_edge][f"m_flow_{load_scenario}"] = 0.0
            graph.edges[normalized_edge][f"contributing_demands_{load_scenario}"] = []
            edges_without_flow += 1
    
    # Step 5: Store supply attribution information at graph level for reference
    graph.graph[f"supply_attribution_{load_scenario}"] = supply_attribution
    
    # Step 6: Calculate and log comprehensive summary statistics
    total_demand_flow = sum(demand_mass_flows.values())
    active_edges_flow = sum(final_edge_flows.values()) if final_edge_flows else 0.0
    
    logger.info(f"Flow aggregation completed successfully:")
    logger.info(f"  - Total demand flow: {total_demand_flow:.6f} kg/s")
    logger.info(f"  - Edges with flow: {edges_with_flow}/{len(graph.edges)}")
    logger.info(f"  - Edges without flow: {edges_without_flow}")
    logger.info(f"  - Supply-demand pairs processed: {len(supply_demand_paths)}")
    
    # Log flow distribution statistics for active edges
    if final_edge_flows:
        max_edge_flow = max(final_edge_flows.values())
        min_edge_flow = min(final_edge_flows.values())
        avg_edge_flow = sum(final_edge_flows.values()) / len(final_edge_flows)
        
        logger.info(f"Active edge flow statistics:")
        logger.info(f"  - Maximum edge flow: {max_edge_flow:.6f} kg/s")
        logger.info(f"  - Minimum edge flow: {min_edge_flow:.6f} kg/s")
        logger.info(f"  - Average edge flow: {avg_edge_flow:.6f} kg/s")
    
    logger.info("Robust edge flow aggregation completed successfully")

def __normalize_edge(edge):
    """
    Normalize edge tuple to consistent order for undirected graph operations.
    
    This function ensures that edges (A, B) and (B, A) are treated as the same edge
    by always returning the tuple with the smaller node ID first.
    
    Args:
        edge (tuple): Edge as (node1, node2)
        
    Returns:
        tuple: Normalized edge as (min_node, max_node)
    """
    return tuple(sorted(edge))

In [ ]:
import logging
import warnings
from typing import Any, List, Optional, Dict

import networkx as nx


def estimate_m_flow(
    graph: Any,
    dT: Optional[float] = None,
    network_type: str = "heating",
    demand_attribute: str = "input_heat",
    load_scenario: str = "peak_load",
    cp: float = 4000,
    logger: Optional[logging.Logger] = None
) -> Any:
    """
    Estimates the mass flow for each edge in the graph based on building loads.
    
    This function calculates the maximum load for each edge in the graph based on
    the shortest path from producer to consumer nodes and then calculates the
    mass flow for each edge based on the maximum load.
    
    Parameters
    ----------
    graph : UESGraph or nx.Graph
        The graph representing the network.
    dT : float, optional
        Temperature difference between supply and return in K.
        If None, will attempt to read from demand nodes' 'dT_Network' attribute.
    network_type : str, optional
        Type of network, default is "heating".
    demand_attribute : str, optional
        Key for the load in the node attributes, default is "input_heat".
    load_scenario : str, optional
        Load scenario, default is "peak_load". Can be "peak_load" or "average_load".
    cp : float, optional
        Specific heat capacity of the fluid in J/(kg*K), default is 4000.
    logger : logging.Logger, optional
        Logger to use for logging messages. If None, creates a new logger.
        
    Returns
    -------
    graph : UESGraph or nx.Graph
        The input graph with additional edge attributes:
        - load_{load_scenario}: Maximum load for the edge
        - load_{load_scenario}_supplies: Dictionary with loads for each producer
        - m_flow_{load_scenario}: Mass flow for the edge
        
    Raises
    ------
    TypeError
        If graph is not a UESGraph object with nodelist_building attribute.
    ValueError
        If network_type is not 'heating' or 'cooling'.
        If load_scenario is not 'peak_load' or 'average_load'.
        If no producer or consumer nodes are found.
        If dT is not provided and cannot be determined from node attributes.
    """
    # Set up logger
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Validate parameters
    if not hasattr(graph, "nodelist_building"):
        raise TypeError("graph must be a UESGraph object with nodelist_building attribute")
    
    if network_type not in ["heating", "cooling"]:
        raise ValueError("network_type must be 'heating' or 'cooling'")
    
    if load_scenario not in ["peak_load", "average_load"]:
        raise ValueError("load_scenario must be 'peak_load' or 'average_load'")
    
    # Find consumer and producer nodes
    demands = []
    supplies = []
    dT_node_values = {}  # Dictionary to store dT values for each demand node

    for node in graph.nodelist_building:
        supply_attr = f"is_supply_{network_type}"
        if supply_attr not in graph.nodes[node]:
            raise ValueError(f"Node {node} does not have attribute '{supply_attr}'")
        
        if graph.nodes[node][supply_attr]:
            supplies.append(node)
        else:
            if demand_attribute not in graph.nodes[node]:
                raise ValueError(f"Node {node} does not have attribute '{demand_attribute}'")
            demands.append(node)
            
            # Collect dT values from demand nodes if available
            if dT is None and 'dT_Network' in graph.nodes[node]:
                dT_node_values[node] = graph.nodes[node]['dT_Network']
    
    if not supplies:
        raise ValueError(f"No producer nodes found for network type '{network_type}'")
    
    if not demands:
        raise ValueError(f"No consumer nodes found for network type '{network_type}'")
    
    # Determine global dT or flag for node-specific usage
    use_node_specific_dT = False

    # Determine temperature difference (dT)
    if dT is not None:
        # Use the explicitly provided dT parameter
        logger.info(f"Using provided dT value: {dT} K")
    elif dT_node_values:
        # We have dT values from nodes, but need to decide how to use them
        if len(set(dT_node_values.values())) == 1:
             # All nodes have the same dT value, so use that
            dT = next(iter(dT_node_values.values()))
            logger.info(f"Using uniform dT={dT} K from all demand nodes")
        elif  len(dT_node_values) == len(demands):
            use_node_specific_dT = True
            # Nodes have different dT values - we'll use node-specific values later
            logger.info(f"Using node-specific dT values from demand nodes")
        else:
            # Some nodes have dT values, but not all - use the average
            dT = sum(dT_node_values.values()) / len(dT_node_values)
            logger.info(f"Using average dT={dT} K from demand nodes")
           
    elif hasattr(graph, "graph") and "dT_design" in graph.graph:
        # Fallback to deprecated attribute
        dT = graph.graph["dT_design"]
        warnings.warn(
            "'dT_design' attribute is deprecated. Use 'dT_Network' in demand nodes instead.",
            DeprecationWarning
        )
        logger.warning(f"Using deprecated 'dT_design' attribute: {dT} K")
    else:
        # Last resort: default value with warning
        dT = 20.0  # Common default for heating networks
        warnings.warn(
            f"No dT specified. Using default value of {dT} K. "
            "Specify dT explicitly or add 'dT_Network' to demand nodes.",
            UserWarning
        )
        logger.warning(f"No dT specified. Using default value of {dT} K")
    
    logger.info(f"Estimating mass flows for {network_type} network with {load_scenario} scenario (dT={dT}K)")
    
    # Calculate loads for edges
    edges_loads = {}
    
    for supply in supplies:
        for demand in demands:
            if nx.has_path(graph, supply, demand):
                # Calculate load for the consumer node
                load_values = [abs(x) for x in graph.nodes[demand][demand_attribute]]
                
                if load_scenario == "peak_load":
                    load = max(load_values)
                elif load_scenario == "average_load":
                    load = sum(load_values) / len(load_values)
                
                # Distribute load along the path
                path = nx.shortest_path(graph, supply, demand)
                for i in range(len(path) - 1):
                    edge = (path[i], path[i + 1])
                    if edge not in edges_loads:
                        edges_loads[edge] = {}
                    edges_loads[edge][supply] = edges_loads[edge].get(supply, 0) + load
    
    # Calculate maximum load for each edge and update the graph
    for edge in edges_loads:
        max_load = max(edges_loads[edge].values())
        graph.edges[edge][f"load_{load_scenario}"] = max_load
        graph.edges[edge][f"load_{load_scenario}_supplies"] = edges_loads[edge]  # for documentation
    
    # Calculate mass flows
    for edge in graph.edges:
        if f"load_{load_scenario}" in graph.edges[edge]:
            load = graph.edges[edge][f"load_{load_scenario}"]
            m_flow = load / (cp * dT)
            graph.edges[edge][f"m_flow_{load_scenario}"] = m_flow
    
    return graph


    """
    Sizes a hydraulic network by estimating mass flows and pipe diameters.
    
    This function is a helper function that calls estimate_m_flow and estimate_pipe_diameter
    in sequence to size a hydraulic network.
    
    Parameters
    ----------
    graph : UESGraph
        The graph representing the network.
    dT : float, optional
        Temperature difference between supply and return in K.
        If None, will attempt to read from demand nodes' 'dT_Network' attribute.
    network_type : str, optional
        Type of network, default is "heating".
    demand_attribute : str, optional
        Key for the load in the node attributes, default is "input_heat".
    load_scenario : str, optional
        Load scenario, default is "peak_load". Can be "peak_load" or "average_load".
    cp : float, optional
        Specific heat capacity of the fluid in J/(kg*K), default is 4000.
    dp_set : float, optional
        Specific pressure loss in Pa/m, default is 100.
    diameters : List[float], optional
        List of available pipe diameters in m.
    logger : logging.Logger, optional
        Logger to use for logging messages. If None, creates a new logger.
        
    Returns
    -------
    graph : UESGraph
        The input graph with additional edge attributes for mass flows and pipe diameters.
        
    Raises
    ------
    TypeError
        If graph is not a UESGraph object.
    """
    

## Archive

In [ ]:
#estimate_m_flow(graph, dT,network_type, key, load_scenario = "peak_load")
import networkx as nx

network_type = "heating"
load_scenario = "peak_load"
key = "input_heat"
demands = []
supplies = []
for node in graph.nodelist_building:
    if graph.nodes[node][f"is_supply_{network_type}"]:
        supplies.append(node)
    else:
        demands.append(node)

edges_loads = {}
for supply in supplies:
    for demand in demands:
        if nx.has_path(graph,supply,demand):
            if load_scenario == "peak_load":
                load = max([abs(x) for x in graph.nodes[node][key]])
                path = nx.shortest_path(graph,supply,demand)
                for i in range(len(path)-1):
                    edge = (path[i],path[i+1])
                    if edge not in edges_loads:
                        edges_loads[edge] = {}
                    edges_loads[edge][supply] = edges_loads[edge].get(supply,0) + load
   

for edge in edges_loads:
    max_load = max(edges_loads[edge].values())
    graph.edges[edge][f"load_{load_scenario}"] = max_load
    graph.edges[edge][f"load_{load_scenario}_supplies"] = edges_loads[edge] # just for documentation
    
cp = 4000
dT = 30
for edge in graph.edges:
    if f"load_{load_scenario}" in graph.edges[edge]:
        load = graph.edges[edge][f"load_{load_scenario}"]
        m_flow = load/(cp*dT)
        graph.edges[edge][f"m_flow_{load_scenario}"] = m_flow